<a href="https://colab.research.google.com/github/fidan-c/human-value-detection/blob/main/Val_Det3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np


In [2]:
random.seed(64)
np.random.seed(64)

In [ ]:
# using Conc and Premise as one representation and Stance added to the labels: bert-base-uncased [SEP] added when concatenating the two columns to one column, lowercased.
!pip install datasets evaluate transformers[sentencepiece]
!pip install transformers
!pip install tokenizer
!pip install simpletransformers

In [4]:

import tensorflow as tf
from transformers import AutoTokenizer,TFAutoModel, DataCollatorWithPadding
import pandas as pd


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [9]:
# load data - not including chinese
# sep for tsv
arguments_training_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-training.tsv', sep='\t') 
arguments_validation_no_label = pd.read_csv('/content/drive/MyDrive/transformers/arguments-validation.tsv', sep='\t')
arguments_test_data = pd.read_table('/content/drive/MyDrive/transformers/arguments-test.tsv')
arguments_test_data_nahjalbalagha= pd.read_table('/content/drive/MyDrive/transformers/arguments-test-nahjalbalagha.tsv')

labels_training = pd.read_csv('/content/drive/MyDrive/transformers/labels-training.tsv', sep='\t').drop('Argument ID', axis=1)
labels_validation= pd.read_csv('/content/drive/MyDrive/transformers/labels-validation.tsv', sep='\t').drop('Argument ID', axis=1)

print(labels_training.shape)
labels_training.head()


#new added label

label_to_add = ["Stance"]

# fixing the dataset for inconsistency: in favor of, in favour of
arguments_training_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)


arguments_test_data['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favour of'],
                       ['in favor of'], inplace=True)

# from categorical value to numerical value for Stance

arguments_training_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_validation_no_label['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)


arguments_test_data['Stance'].replace(['in favor of', 'against'],
                       [1, 0], inplace=True)

arguments_test_data_nahjalbalagha['Stance'].replace(['in favor of','against'],
                                                    [1,0], inplace=True)



(5393, 20)


In [10]:
labels_training['Stance_As_Label'] = arguments_training_no_label[label_to_add] 
print(labels_training.shape)
print(labels_training.head())



labels_validation['Stance_As_Label'] = arguments_validation_no_label[label_to_add]
print(labels_validation.shape)
print(labels_validation.head())


(5393, 21)
   Self-direction: thought  Self-direction: action  Stimulation  Hedonism  \
0                        0                       0            0         0   
1                        0                       0            0         0   
2                        0                       0            0         0   
3                        0                       0            0         0   
4                        0                       0            0         0   

   Achievement  Power: dominance  Power: resources  Face  Security: personal  \
0            0                 0                 0     0                   0   
1            0                 0                 0     0                   1   
2            0                 1                 0     0                   0   
3            0                 0                 0     0                   0   
4            0                 0                 0     0                   1   

   Security: societal  ...  Conformity: rules

In [ ]:
arguments_test_data_nahjalbalagha.head()

In [12]:
# training set
train_labels = labels_training.values.tolist()

cols_to_concat = ["Conclusion", "Premise"]

arguments_training_no_label['Concated'] = arguments_training_no_label[cols_to_concat].apply(lambda x: ' [SEP] '.join(x.astype(str)),
    axis=1)

print(arguments_training_no_label['Concated'][0])

#lowercase every word - roberta is case sensitive, thats why

print(arguments_training_no_label['Concated'][0])

train_text = arguments_training_no_label['Concated'].values
train_data = [[t, l] for t, l in zip(train_text, train_labels)] 
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]


We should ban human cloning [SEP] we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.
We should ban human cloning [SEP] we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.


In [13]:

# repeat above for the validation set
val_labels = labels_validation.values.tolist()

arguments_validation_no_label['Concated'] = arguments_validation_no_label[cols_to_concat].apply(lambda x: ' [SEP] '.join(x.astype(str)),
    axis=1)
print(arguments_validation_no_label['Concated'][0])


print(arguments_validation_no_label['Concated'][0])


val_text = arguments_validation_no_label['Concated'].values
val_data = [[t, l] for t, l in zip(val_text, val_labels)] 
val_df = pd.DataFrame(val_data)
val_df.columns = ["text", "labels"]




Entrapment should be legalized [SEP] if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?
Entrapment should be legalized [SEP] if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?


In [ ]:
print(val_df.head())
print(val_df.shape)
print(len(val_labels[0]))

In [14]:
# repeat above for the main test set

arguments_test_data['Concated'] = arguments_test_data[cols_to_concat].apply(lambda x: ' [SEP] '.join(x.astype(str)),
    axis=1)

test_data = arguments_test_data['Concated'].values
print(np.shape(test_data))

print(arguments_test_data['Concated'][0])
print(arguments_test_data['Concated'][0])

(1576,)
We should end affirmative action [SEP] affirmative action helps with employment equity.
We should end affirmative action [SEP] affirmative action helps with employment equity.


In [ ]:
# repeat above for the nahjalbalagha test set

arguments_test_data_nahjalbalagha['Concated'] = arguments_test_data_nahjalbalagha[cols_to_concat].apply(lambda x: ' [SEP] '.join(x.astype(str)),
    axis=1)

test_data_nahjalbalagha = arguments_test_data_nahjalbalagha['Concated'].values
print(np.shape(test_data_nahjalbalagha))
print(test_data_nahjalbalagha[0])


print(arguments_test_data_nahjalbalagha['Concated'][0])

In [16]:

model_args = MultiLabelClassificationArgs(num_train_epochs=5, train_batch_size=16, overwrite_output_dir=True)

# Create a MultiLabelClassificationModel from simple transformers: <https://simpletransformers.ai/docs/multi-label-classification/>
model = MultiLabelClassificationModel(
    "bert",
    "bert-large-uncased",
    num_labels=len(labels_training.columns),
    args=model_args,
)

# Train the model
model.train_model(train_df)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMultiLabelSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassification were not

  0%|          | 0/5393 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/338 [00:00<?, ?it/s]

(1690, 0.27626502338862985)

In [17]:
# LRAP, with stance predicted
result, model_outputs, wrong_predictions = model.eval_model(val_df) 

  0%|          | 0/1896 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/237 [00:00<?, ?it/s]

In [18]:
result

{'LRAP': 0.7449091131768033, 'eval_loss': 0.3384019376607887}

In [19]:
from sklearn.metrics import f1_score

In [20]:
# predicted labels and true labels for the validation set for F1 score, without stance

# f1_score functions takes np.array

#text = val_df['text'].to_list()

preds, _ = model.predict(list(val_df['text']))
preds= np.array(preds)
preds_without_last_col = preds[:, :-1]

true_labels =np.array(val_labels)
true_labels_without_last_col=true_labels[:,:-1]


# F1 score for each label separately
f1_scores = f1_score(true_labels_without_last_col, preds_without_last_col, average=None)

# average F1 score across all labels
average_f1_score = np.mean(f1_scores)


  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [21]:
print(preds_without_last_col.shape)
print(true_labels_without_last_col.shape)

(1896, 20)
(1896, 20)


In [22]:
average_f1_score

0.39704656380382014

In [23]:
f1_scores

array([0.51901566, 0.54761905, 0.14285714, 0.25396825, 0.63339731,
       0.23529412, 0.43859649, 0.05594406, 0.74055093, 0.61668427,
       0.41322314, 0.53935185, 0.14925373, 0.02919708, 0.59252971,
       0.16363636, 0.65171898, 0.66666667, 0.10332103, 0.44810544])

In [24]:
# f1 score with Stance prediction included 

preds, _ = model.predict(list(val_df['text']))
preds= np.array(preds)
true_labels =np.array(val_labels)

# F1 score for each label separately
f1_scores = f1_score(true_labels, preds, average=None)

# average F1 score across all labels
average_f1_score = np.mean(f1_scores)

  0%|          | 0/1896 [00:00<?, ?it/s]

  0%|          | 0/237 [00:00<?, ?it/s]

In [25]:
average_f1_score

0.4179293244113832

In [ ]:
# predictions for the main test set
test_preds = model.predict(list(test_data))

In [ ]:
# prediction outputs for the main test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data = []

for arg_id, labels in zip(arguments_test_data["Argument ID"].values, test_preds[0]):
  row = [arg_id] + labels[:]
  pred_data.append(row)

pred_df = pd.DataFrame(pred_data, columns=cols)
pred_df_no_stance = pred_df.drop("Stance_As_Label",axis=1)

pred_df_no_stance.to_csv("main_test_pred.tsv", index=False, sep='\t')

In [ ]:
# predictions for the nahjalbalagha test set
test_preds_nahjalbalagha= model.predict(list(test_data_nahjalbalagha))

In [ ]:
# prediction outputs for the nahjalbalagha test set
cols = ["Argument ID"] + list(labels_validation.columns)
pred_data_nah = []

for arg_id, labels in zip(arguments_test_data_nahjalbalagha["Argument ID"].values, test_preds_nahjalbalagha[0]):
  row = [arg_id] + labels[:]
  pred_data_nah.append(row)

pred_nah_df = pd.DataFrame(pred_data_nah, columns=cols)
pred_df_nah_no_stance = pred_nah_df.drop("Stance_As_Label",axis=1)

pred_df_nah_no_stance.to_csv("nahjalbalagha_test_pred.tsv", index=False, sep='\t')

In [ ]:
pred_df_nah_no_stance.shape

In [ ]:
pred_df_no_stance.shape

In [ ]:
! zip model.zip /content/outputs/pytorch_model.bin

  adding: content/outputs/pytorch_model.bin


zip error: Interrupted (aborting)


In [ ]:
! cp /content/outputs/pytorch_model.bin /content/drive/MyDrive/pytorch_model.bin